In [ ]:
from __future__ import annotations

import logging
from alaska_legislative_data import _ingest, _curated
from nest_asyncio import apply

apply()

logging.getLogger("alaska_legislative_data").setLevel(logging.INFO)
logging.getLogger("alaska_legislative_data").addHandler(logging.StreamHandler())

db = _ingest.get_db()

In [29]:
db.Member.count()

┌──────┐
│ 2088 │
└──────┘

In [30]:
members = _curated.read_members(backend=db)
members

Reading members from /Users/nc/code/scg/alaska-legislative-data/python/alaska_legislative_data/members.csv
Reading members from /Users/nc/code/scg/alaska-legislative-data/python/alaska_legislative_data/members.csv


┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ MemberId               ┃ LegislatureNumber ┃ PersonId          ┃ MemberCode ┃ MemberChamber ┃ MemberDistrict ┃ MemberParty ┃ MemberIsMajority ┃ MemberIsActive ┃ MemberComment ┃ MemberEMail ┃ MemberPhone ┃ MemberBuilding ┃ MemberRoom ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string                 │ int64             │ string            │ string     │ string        │ string         │ string      │ boolean          │ boolean        │ string        │ string      │ int64       │ string         │ string     │
├────────────────────────┼───────────────────┼───────────────────┼────────────┼───────────────┼────────────────┼─────────────┼──────────────────┼────────────────┼───────────────┼─────────────┼─────────────┼────────────────┼────────────┤
│ 1:H::John Rader:1      │                 1 │ John Rader:1      │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ 1:H::Oral Freeman:1    │                 1 │ Oral Freeman:1    │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ 1:H::Russ Meekins:1    │                 1 │ Russ Meekins:1    │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ 1:H::Charles M Jones:1 │                 1 │ Charles M Jones:1 │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ 1:H::Allen Peterson:1  │                 1 │ Allen Peterson:1  │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ 1:H::Andrew Hope:1     │                 1 │ Andrew Hope:1     │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ 1:H::Axel Johnson:1    │                 1 │ Axel Johnson:1    │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ 1:H::Blanche McSmith:1 │                 1 │ Blanche McSmith:1 │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ 1:H::Bob Giersdorf:1   │                 1 │ Bob Giersdorf:1   │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ 1:H::Bruce Kendall:1   │                 1 │ Bruce Kendall:1   │ NULL       │ H             │ NULL           │ NULL        │ NULL             │ NULL           │ NULL          │ NULL        │        NULL │ NULL           │ NULL       │
│ …                      │                 … │ …                 │ …          │ …             │ …              │ …           │ …                │ …              │ …             │ …           │           … │ …              │ …          │
└────────────────────────┴───────────────────┴───────────────────┴────────────┴───────────────┴────────────────┴─────────────┴──────────────────┴────────────────┴───────────────┴─────────────┴─────────────┴──

In [33]:
def update(
    db: SQLBackend,
    table_name: str,
    new_data: ir.Table,
    *,
    database: str | tuple[str, str] | None = None,
    join_key: str | None = None,
) -> ir.Table:
    """
    Update data in a backend's table.

    Parameters
    ----------
    db
        The backend to update.
    table_name
        The name of the table to update.
    new_data
        The new data to update with. You don't have to pass all the columns in the
        existing table, just the ones you want to update.
    database
        The database or (catalog, database) tuple to lookup the existing table in.
    join_key
        The column to join on.
        If None, the primary key of the existing table will be used.

    Returns
    -------
    The updated table.
    """
    existing_table = db.table(table_name, database=database)
    extra = [c for c in new_data.columns if c not in existing_table.columns]
    if extra:
        raise ValueError(f"new data has extra columns: {extra}")
    if join_key is None:
        join_key = get_primary_key(existing_table)
        if join_key is None:
            raise ValueError(
                f"If you don't specify join_key, table {table_name} must have a primary key."  # noqa: E501
            )
    if database is None:
        table_spec = f'"{table_name}"'
    elif isinstance(database, str):
        table_spec = f'"{database}"."{table_name}"'
    else:
        table_spec = f'"{database[0]}"."{database[1]}"."{table_name}"'

    if join_key not in existing_table:
        raise ValueError(f"join key '{join_key}' not found in table '{table_spec}'")
    # This appears to have a bug with over-eager key constraints if you
    # try to update array columns. I'm not sure how to fix it yet.
    template = """
    WITH new AS (
        {new_sql}
    )
    UPDATE {table_spec}
    SET {set_sql}
    FROM new
    WHERE {table_spec}."{join_key}" = new."{join_key}";
    """
    new_sql = db.compile(new_data)
    cols_to_update = [
        c for c in existing_table.columns if c in new_data.columns and c != join_key
    ]
    set_sql = ", ".join(f'"{col}" = new."{col}"' for col in cols_to_update)
    sql = template.format(
        new_sql=new_sql,
        table_spec=table_spec,
        set_sql=set_sql,
        join_key=join_key,
    )
    # logger.info(f"Running SQL:\n{sql}")
    db.raw_sql(sql)
    return existing_table


update(db, "members", members.select("MemberId", "MemberCode"), join_key="MemberId")

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ MemberId               ┃ LegislatureNumber ┃ PersonId          ┃ MemberChamber ┃ MemberDistrict ┃ MemberIsMajority ┃ MemberParty ┃ MemberCode ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ !string                │ !int32            │ !string           │ !string       │ string         │ boolean          │ string      │ string     │
├────────────────────────┼───────────────────┼───────────────────┼───────────────┼────────────────┼──────────────────┼─────────────┼────────────┤
│ 1:H::Oral Freeman:1    │                 1 │ Oral Freeman:1    │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ 1:H::Earl Hillstrand:1 │                 1 │ Earl Hillstrand:1 │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ 1:H::James Hurley:1    │                 1 │ James Hurley:1    │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ 1:H::John Rader:1      │                 1 │ John Rader:1      │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ 1:H::Charles M Jones:1 │                 1 │ Charles M Jones:1 │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ 1:H::Allen Peterson:1  │                 1 │ Allen Peterson:1  │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ 1:H::Andrew Hope:1     │                 1 │ Andrew Hope:1     │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ 1:H::Axel Johnson:1    │                 1 │ Axel Johnson:1    │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ 1:H::Bob Giersdorf:1   │                 1 │ Bob Giersdorf:1   │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ 1:H::Bruce Kendall:1   │                 1 │ Bruce Kendall:1   │ H             │ NULL           │ NULL             │ NULL        │ NULL       │
│ …                      │                 … │ …                 │ …             │ …              │ …                │ …           │ …          │
└────────────────────────┴───────────────────┴───────────────────┴───────────────┴────────────────┴──────────────────┴─────────────┴────────────┘

In [27]:
members.MemberCode.topk()

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ MemberCode ┃ MemberCode_count ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ string     │ int64            │
├────────────┼──────────────────┤
│ NULL       │              555 │
│ HOF        │               18 │
│ OLS        │               13 │
│ ELL        │               13 │
│ SAN        │               12 │
│ STG        │               12 │
│ SMN        │               12 │
│ WIE        │               10 │
│ KER        │               10 │
│ FOS        │               10 │
│ …          │                … │
└────────────┴──────────────────┘

In [34]:
_ingest.ingest_members(db)

Reading members from /Users/nc/code/scg/alaska-legislative-data/python/alaska_legislative_data/members.csv
Reading members from /Users/nc/code/scg/alaska-legislative-data/python/alaska_legislative_data/members.csv


Ingesting 2088 members
Ingesting 2088 members
Found 2088 existing members
Found 2088 existing members
Found 0 new members
Found 0 new members


In [35]:
_ingest.ingest_votes_and_choices(db)

Desired leg_nums: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Desired leg_nums: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Existing leg_nums: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Existing leg_nums: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Missing leg_nums: [34]
Missing leg_nums: [34]
Scraping missing votes for [(34, 'STO'), (34, 'HNN'), (34, 'STS'), (34, 'RUF'), (34, 'ELA'), (34, 'HLA'), (34, 'KOP'), (34, 'COU'), (34, 'SCG'), (34, 'JOS'), (34, 'GAL'), (34, 'FIE'), (34, 'NED'), (34, 'MNA'), (34, 'GRY'), (34, 'BYN'), (34, 'HMS'), (34, 'VAN'), (34, 'COS'), (34, 'HLL'), (34, 'MEA'), (34, 'EIS'), (34, 'ALR'), (34, 'JHS'), (34, 'TLT'), (34, 'UND'), (34, 'MOO'), (34, 'RAU'), (34, 'MCA'), (34, 'DIB'), (34, 'SPP'), (34, 'PRA'), (34, 'TMZ'), (34, 'CAI'), (34, 'SCW'), (34, 'EDG'), (34, 'JIM'), (34, 'FON'), (34, 'BUR'), (34, 'SMN'), (34, 'KIE'), (34, 'STG'), (34, 'BJK'), (34, 'GIE'), (34, 'KAU'), (34, 'G

RuntimeError: asyncio.run() cannot be called from a running event loop

In [4]:
import duckdb

con = duckdb.connect()
con.sql("""
    CREATE TYPE mood AS ENUM ('sad', 'ok', 'happy');
    CREATE TABLE person (
        name text,
        current_mood mood
    );
    INSERT INTO person VALUES ('Pedro','happy'), ('Mark', NULL), ('Pagliacci', 'sad'), ('Mr. Mackey', 'ok');
    SELECT * FROM person;
""")

┌────────────┬────────────────────────────┐
│    name    │        current_mood        │
│  varchar   │ enum('sad', 'ok', 'happy') │
├────────────┼────────────────────────────┤
│ Pedro      │ happy                      │
│ Mark       │ NULL                       │
│ Pagliacci  │ sad                        │
│ Mr. Mackey │ ok                         │
└────────────┴────────────────────────────┘

In [8]:
db2 = ibis.duckdb.from_connection(con)
moods = db2.table("person").select("current_mood")
at2 = db2._to_duckdb_relation(moods).arrow()
at2

pyarrow.Table
current_mood: dictionary<values=string, indices=uint8, ordered=0>
----
current_mood: [  -- dictionary:
["sad","ok","happy"]  -- indices:
[2,null,0,1]]

In [9]:
moods.execute()

,current_mood
0,happy
1,None
2,sad
3,ok


In [10]:
at2.column(0).to_pandas()

ArrowTypeError: Converting unsigned dictionary indices to pandas not yet supported, index type: uint8

In [14]:
db.sql("SELECT Chamber from members").to_pandas()

ArrowTypeError: Converting unsigned dictionary indices to pandas not yet supported, index type: uint8

In [12]:
db.con.sql("SELECT Chamber from members")

┌────────────────┐
│    Chamber     │
│ enum('h', 's') │
├────────────────┤
│ H              │
│ H              │
│ H              │
│ H              │
│ H              │
│ S              │
│ H              │
│ S              │
│ H              │
│ H              │
│ ·              │
│ ·              │
│ ·              │
│ H              │
│ H              │
│ H              │
│ S              │
│ H              │
│ H              │
│ H              │
│ H              │
│ H              │
│ H              │
├────────────────┤
│   1472 rows    │
│   (20 shown)   │
└────────────────┘

In [50]:
at = db.con.sql("SELECT Chamber from members").arrow()
at

pyarrow.Table
Chamber: dictionary<values=string, indices=uint8, ordered=0>
----
Chamber: [  -- dictionary:
["H","S"]  -- indices:
[0,0,0,0,0,...,0,0,0,0,0]]

In [30]:
import sqlglot as sg
import sqlglot.expressions as sge

sgtype = sg.parse_one("ENUM('sad', 'ok', 'happy')", into=sge.DataType, read="postgres")
sgtype

DataType(
  this=Type.ENUM,
  expressions=[
    Literal(this=sad, is_string=True),
    Literal(this=ok, is_string=True),
    Literal(this=happy, is_string=True)],
  nested=False)

In [31]:
sgtype = sg.parse_one("enum('sad', 'ok', 'happy')", into=sge.DataType, read="postgres")
sgtype

DataType(
  this=Type.ENUM,
  expressions=[
    Literal(this=sad, is_string=True),
    Literal(this=ok, is_string=True),
    Literal(this=happy, is_string=True)],
  nested=False)

In [34]:
db._to_duckdb_relation(db.Member)

┌─────────────────────────────────┬───────────────────┬────────────────────────┬────────────────┬──────────┬────────────┬─────────┐
│            MemberId             │ LegislatureNumber │        PersonId        │    Chamber     │ District │ IsMajority │  Party  │
│             varchar             │       int32       │        varchar         │ enum('h', 's') │ varchar  │  boolean   │ varchar │
├─────────────────────────────────┼───────────────────┼────────────────────────┼────────────────┼──────────┼────────────┼─────────┤
│ 13:H:20-A:Robert Bettisworth:11 │                13 │ Robert Bettisworth:11  │ H              │ 20-A     │ false      │ R       │
│ 11:H:13:Patrick O'Connell:11    │                11 │ Patrick O'Connell:11   │ H              │ 13       │ false      │ R       │
│ 11:H:18:Vernon Hurlbert:11      │                11 │ Vernon Hurlbert:11     │ H              │ 18       │ false      │ D       │
│ 11:H:20:Brian Rogers:11         │                11 │ Brian Rogers:11     

In [46]:
db.con.sql("SELECT Chamber from members").arrow().column(0).to_pandas()

ArrowTypeError: Converting unsigned dictionary indices to pandas not yet supported, index type: uint8

In [41]:
db.Member.select(
    # "MemberId",
    # "LegislatureNumber",
    # "PersonId",
    "Chamber",
    # "District",
    # "IsMajority",
    # "Party",
).to_pandas()

ArrowTypeError: Converting unsigned dictionary indices to pandas not yet supported, index type: uint8